<a href="https://colab.research.google.com/github/KevinMarcos26/DataScienceMundial2022/blob/main/DataScienceMundial2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PROYECTO DATA SCIENCE 2022 WORLD CUP QATAR**

EXTRACCIÓN DE DATOS

In [15]:
import pandas as pd
from string import ascii_uppercase as alfa
import pickle

In [16]:
tablas_mundial = pd.read_html("https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup")

In [ ]:
tablas_mundial[19].columns

In [ ]:
tablas_mundial[61]

In [ ]:
alfa

In [ ]:
tablas_mundial = pd.read_html("https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup")

dict = {}
for letra, i in zip(alfa,range(12,68,7)):
  df = tablas_mundial[i]
  df.rename(columns={df.columns[1]: 'Team'}, inplace= True)
  df.pop("Qualification")
  dict[f'Grupo {letra}'] = df
  print(df)


In [18]:
dict.keys()

dict_keys(['Grupo A', 'Grupo B', 'Grupo C', 'Grupo D', 'Grupo E', 'Grupo F', 'Grupo G', 'Grupo H'])

In [ ]:
dict['Grupo H']

In [19]:
with open('dict', 'wb') as output:
   pickle.dump(dict,output)

In [ ]:
pip install bs4

In [ ]:
pip install lxml

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas
import lxml


years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

def get_matches(year):
  web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
  response = requests.get(web)
  content = response.text
  soup = BeautifulSoup(content,'lxml')
  matches = soup.find_all('div',class_='footballbox')

  home = []
  score = []
  away = []

  for match in matches:
     home.append(match.find('th', class_='fhome').get_text)
     score.append(match.find('th', class_='fscore').get_text)
     away.append(match.find('th', class_='faway').get_text)

     dict_football = {'home': home, 'score': score, 'away': away}
     df_football = pd.DataFrame(dict_football)
     df_football['year'] = year
     return df_football

fifa = [get_matches(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv("fifa_worldcup_historical_data.csv", index=False)

# fixture
df_fixture = get_matches(2022)
df_fixture.to_csv('fifa_worldcup_fixture.csv', index=False)


In [ ]:
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

path = "content/chromedriver.exe"
service = Service(executable_path = path)
driver = webdriver.Chrome(service = service)

def get_misssing_data(year):
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'

    driver.get(web)
    matches = driver.find_elements(by='xpath', value='//td[@align="right"]/.. | //td[@style="text-align:right;"]/..')
    # matches = driver.find_elements(by='xpath', value='//tr[@style="font-size:90%"]')

    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find_element(by='xpath', value='./td[1]').text)
        score.append(match.find_element(by='xpath', value='./td[2]').text)
        away.append(match.find_element(by='xpath', value='./td[3]').text)

    dict_football = {'home': home, 'score': score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    time.sleep(2)
    return df_football


years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

fifa = [get_misssing_data(year) for year in years]
driver.quit()
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv("fifa_worldcup_missing_data.csv", index=False)



**LIMPIEZA DATOS**

In [8]:
import pandas as pd

In [9]:
df_historical_data = pd.read_csv('/content/fifa_worldcup_historical_data.csv')
df_fixture = pd.read_csv('/content/fifa_worldcup_fixture.csv')
df_missing_data = pd.read_csv('/content/fifa_worldcup_missing_data.csv')

In [ ]:
df_fixture

In [11]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

In [ ]:
df_missing_data.dropna(inplace=True)
df_historical_data = pd.concat([df_historical_data, df_missing_data], ignore_index=True)
df_historical_data.drop_duplicates(inplace=True)
df_historical_data.sort_values('year', inplace=True)
df_historical_data

In [ ]:
# deleting match with walk over
delete_index = df_historical_data[df_historical_data['home'].str.contains('Sweden') &
                                  df_historical_data['away'].str.contains('Austria')].index

df_historical_data.drop(index=delete_index, inplace=True)

# cleanning score and home/away columns
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)
df_historical_data['home'] = df_historical_data['home'].str.strip() # clean blank spaces: Yugoslavia twice
df_historical_data['away'] = df_historical_data['away'].str.strip()

# splitting score columns into home and away goals and dropping score column
df_historical_data[['HomeGoals', 'AwayGoals']] = df_historical_data['score'].str.split('–', expand=True)
df_historical_data.drop('score', axis=1, inplace=True)

# renaming columns and changing data types
df_historical_data.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam', 
                                   'year':'Year'}, inplace=True)
df_historical_data = df_historical_data.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

# creating new column "totalgoals"
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']
df_historical_data

In [14]:
df_historical_data.to_csv('clean_fifa_worldcup_matches.csv',index=False)
df_fixture.to_csv('clean_fifa_worldcup_fixture.csv',index=False)

**MODELO DE PROBABILIDAD**

In [20]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [21]:
dict_table = pickle.load(open('dict','rb'))
df_historical_data = pd.read_csv('/content/clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('/content/clean_fifa_worldcup_fixture.csv')

In [ ]:
dict.keys()

In [23]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


In [24]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)


In [48]:
print(predict_points('Argentina','Brazil'))

(0.961485596233405, 1.845703399768912)


In [26]:
print(predict_points('England','United States'))

(2.2356147635326007, 0.5922397535606193)


In [31]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [40]:
for group in dict:
    teams_in_group = dict[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict[group].loc[dict[group]['Team'] == home, 'Pts'] += points_home
        dict[group].loc[dict[group]['Team'] == away, 'Pts'] += points_away

    dict[group] = dict[group].sort_values('Pts', ascending=False).reset_index()
    dict[group] = dict[group][['Team', 'Pts']]
    dict[group] = dict[group].round(0)


In [ ]:
dict['Grupo C']